In [5]:
import numpy as np 
import random 
import os 
path = "./CONTCAR"
topath = "./RWCS_test"
import os
try:
    os.makedirs(topath)
except:
    print("already done")
class pos:
    def __init__(self, fname="./POSCAR",type="auto",para=None):
        self.raw_data = [iline.strip().split() for iline in open(fname,'r').read().strip().split("\n")]
        self.mat_cell = np.array(self.raw_data[2:5], dtype=float)
        self.mat_frac = float(self.raw_data[1][0])
        self.mat_cell = self.mat_frac*self.mat_cell
        self.inv_cell = np.linalg.inv(self.mat_cell)
        self.lst_atom = np.array(self.raw_data[6],dtype=int)
        self.tot_atom = np.sum(self.lst_atom) 
        self.lst_atyp = self.raw_data[5] 
        self.loc_atom = np.array(self.raw_data[8:8+self.tot_atom],dtype=float)
        if "D" in self.raw_data[7][0]:
            self.frc_cord = self.loc_atom.copy()
            self.loc_atom = np.matmul(self.frc_cord, self.mat_cell) 
        else :
            self.frc_cord = np.matmul(self.loc_atom, self.inv_cell)

        if type =="auto":
            self.max_step = 1.0 ##1.0 å
        elif type=="opt":
            self.max_step = para  ##opt param space
    def mutation(self):
        tmv = np.random.RandomState().normal(scale=self.max_step,size=(self.tot_atom,3))
        self.loc_atom = self.loc_atom + tmv 
        self.frc_cord = np.matmul(self.loc_atom, self.inv_cell)
    def opt_mutation(self):
        tmv = []
        #print(self.max_step)
        for x in range(self.tot_atom):     
            #print(self.max_step[0][x])
            tmr = self.max_step[0][x]*float(np.random.rand(1))   ## para = [para_r,para_s,para_p]
            phi = self.max_step[1][x]
            sei = self.max_step[2][x]
            print(tmr,phi,sei)
            tmx = float(tmr*np.sin(phi)*np.cos(sei))
            tmy = float(tmr*np.sin(phi)*np.sin(sei))
            tmz = float(tmr*np.cos(phi))
            tms = [tmx,tmy,tmz]
            #print(x,tms)
            #print("-"*6)
            tmv.append(tms)
        self.loc_atom = self.loc_atom + tmv 
        self.frc_cord = np.matmul(self.loc_atom, self.inv_cell)
    def output(self, oname="./POSCAR"):
        with open(oname, 'w') as f:
            print(self.raw_data[0][0],file=f)
            print("1.00000",file=f)
            [print("  {0[0]:16.12f}  {0[1]:16.12f}  {0[2]:16.12f}".format(i), file=f) for i in self.mat_cell]
            print(" ".join(self.lst_atyp),file=f)
            print(" ".join(["{}".format(ia) for ia in self.lst_atom]),file=f)
            print("Direct", file=f)
            print("\n".join(["  {0[0]:16.12f}  {0[1]:16.12f}  {0[2]:16.12f}".format(i) for i in self.frc_cord]), file=f)
            
        

already done


In [7]:
    
########################################################################
para = np.loadtxt('para.txt').tolist()
para = [[1,20,10000,1,1,1,1,1],
        [1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1]]
_ = pos("../CONTCAR",type="opt",para=para)
_.opt_mutation()
_.output("./POSCAR")

########################################################################

0.12517725393730683 1 1
6.908030234577787 1 1
2250.85453133529 1 1
0.22650330096881566 1 1
0.45976518090818097 1 1
0.8612573305646956 1 1
0.5225231040132795 1 1
0.9980871341452974 1 1


In [3]:
elem_cova = {'H': 31.0,'He': 28.0,'Li': 128.0, 'Be': 96.0, 'B': 84.0, 'C': 76.0, 'N': 71.0, 'O': 66.0, 'F': 57.0, 'Ne': 58.0, 'Na': 166.0,
         'Mg': 141.0, 'Al': 121.0, 'Si': 111.0, 'P': 107.0, 'S': 105.0, 'Cl': 102.0, 'Ar': 106.0, 'K': 203.0, 'Ca': 176.0,'Sc': 170.0, 
         'Ti': 160.0,'V': 153.0, 'Cr': 139.0, 'Mn': 139.0, 'Fe': 132.0, 'Co': 126.0, 'Ni': 124.0, 'Cu': 132.0, 'Zn': 122.0, 'Ga': 122.0, 
         'Ge': 120.0, 'As': 119.0, 'Se': 120.0, 'Br': 120.0, 'Kr': 116.0, 'Rb': 220.0, 'Sr': 195.0, 'Y': 190.0, 'Zr': 175.0, 
         'Nb': 164.0, 'Mo': 154.0, 'Tc': 147.0, 'Ru': 146.0, 'Rh': 142.0, 'Pd': 139.0, 'Ag': 145.0, 'Cd': 144.0, 'In': 142.0, 
         'Sn': 139.0, 'Sb': 139.0, 'Te': 138.0, 'I': 139.0, 'Xe': 140.0, 'Cs': 244.0, 'Ba': 215.0, 'La': 207.0, 'Ce': 204.0, 
         'Pr': 203.0, 'Nd': 201.0, 'Pm': 199.0, 'Sm': 198.0, 'Eu': 198.0, 'Gd': 196.0, 'Tb': 194.0, 'Dy': 192.0, 'Ho': 192.0,
         'Er': 189.0, 'Tm': 190.0, 'Yb': 187.0, 'Lu': 187.0, 'Hf': 175.0, 'Ta': 170.0, 'W': 162.0, 'Re': 151.0, 'Os': 144.0,
         'Ir': 141.0, 'Pt': 136.0, 'Au': 136.0, 'Hg': 132.0, 'Tl': 145.0, 'Pb': 146.0, 'Bi': 148.0,'Po': 140.0, 'At': 150.0, 
         'Rn': 150.0, 'Fr': 260.0, 'Ra': 221.0, 'Ac': 215.0, 'Th': 206.0, 'Pa': 200.0, 'U': 196.0, 'Np': 190.0, 'Pu': 187.0,
         'Am': 180.0, 'Cm': 169.0}
cova = []
_ = pos("./CONTCAR",type="opt",para=para)
for i in _.lst_atyp:
    #print(i,elem_cova[str(i)]/100)
    cova.append(elem_cova[str(i)]/100)
cova.sort()
rw_req = cova[0]+cova[1]
flag = 0
rw_req = rw_req
s = 1
while flag == 0:
    _ = pos("./CONTCAR")
    _.opt_mutation()
    rw_loc = _.loc_atom
    rw_dis = []
    for i in range(len(rw_loc)-1):
        for j in range(i):
            rw_distance = (rw_loc[i][0] - rw_loc[i+1][0])**2 + (rw_loc[i][1]-rw_loc[i+1][1])**2 + (rw_loc[i][2]-rw_loc[i+1][2])**2
            #print("a,b:",i,j,rw_distance)
            rw_dis.append(rw_distance)
    rw_min = min(rw_dis)
    s = s+1
    print("test_",s)
    if rw_min > rw_req:
        flag = 1
        _.output("./POSCAR")
        _.output(topath+"/POSCAR")

already done


TypeError: 'float' object is not subscriptable

In [ ]:
para = [r_list,p_list,s_list]

In [8]:
para = [[1,2,3,4,1,1,1,1],
        [1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1]]

np.savetxt('para.txt',para)

In [24]:
b=np.loadtxt('para.txt').tolist()

In [27]:
b

[[1.0, 20.0, 10000.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]

In [11]:
_ = pos("../CONTCAR",type="opt",para=b)
_.opt_mutation()
_.output("./POSCAR")

NameError: name 'pos' is not defined

In [10]:
ls

band.conf*      goRW.sh*  loop.py*      __pycache__/  RW_old2.py*
debug.py*       INCAR01*  pho_INCAR*    RW_break.py   RW_old.py*
debug_rw.ipynb  INCAR02*  pho_KPOINTS*  RW_bys.py     svasp.lsf*
ener_per.py*    INCAR03*  POSCAR        RWCS_test/


In [11]:
cat POSCAR

from_pyxtal
1.00000
    4.619639827485    0.000000000000    0.000000000000
   -0.000000000000    4.619639827485    0.000000000000
    0.000000000000   -0.000000000000    4.619639827485
Zn Mg O
3 1 4
Direct
    0.005294560034    0.508245788696    0.506292029232
    1.528112855706    1.601190903294    1.721804285909
  849.227934243600  1322.315440921380  1008.624123192269
    0.077969309099    0.121430004276    0.092658345334
    0.322357224835    0.363708673518    0.336333368695
    0.776127118585    0.789671612043    0.277048848524
    0.763125754712    0.265767090194    0.765254188652
    0.289157631084    0.815659467077    0.800535231290


In [1]:
import numpy as np 
import random 
import os 
path = "./CONTCAR"
topath = "./before_RW"
import os
try:
    os.makedirs(topath)
except:
    print("already done")


class pos:
	def __init__(self, fname="./POSCAR"):
		self.raw_data = [iline.strip().split() for iline in open(fname,'r').read().strip().split("\n")]
		self.mat_cell = np.array(self.raw_data[2:5], dtype=float)
		self.mat_frac = float(self.raw_data[1][0])
		self.mat_cell = self.mat_frac*self.mat_cell
		self.inv_cell = np.linalg.inv(self.mat_cell)
		self.lst_atom = np.array(self.raw_data[6],dtype=int)
		self.tot_atom = np.sum(self.lst_atom) 
		self.lst_atyp = self.raw_data[5] 
		self.loc_atom = np.array(self.raw_data[8:8+self.tot_atom],dtype=float)
		if "D" in self.raw_data[7][0]:
			self.frc_cord = self.loc_atom.copy()
			self.loc_atom = np.matmul(self.frc_cord, self.mat_cell) 
		else :
			self.frc_cord = np.matmul(self.loc_atom, self.inv_cell)
		self.max_step = 1.0 ##1.0 å
	
	def mutation(self):
		tmv = np.random.RandomState().normal(scale=self.max_step,size=(self.tot_atom,3))
		self.loc_atom = self.loc_atom + tmv 
		self.frc_cord = np.matmul(self.loc_atom, self.inv_cell)

	def output(self, oname="./POSCAR"):
		with open(oname, 'w') as f:
			print(self.raw_data[0][0],file=f)
			print("1.00000",file=f)
			[print("  {0[0]:16.12f}  {0[1]:16.12f}  {0[2]:16.12f}".format(i), file=f) for i in self.mat_cell]
			print(" ".join(self.lst_atyp),file=f)
			print(" ".join(["{}".format(ia) for ia in self.lst_atom]),file=f)
			print("Direct", file=f)
			print("\n".join(["  {0[0]:16.12f}  {0[1]:16.12f}  {0[2]:16.12f}".format(i) for i in self.frc_cord]), file=f)

already done


In [6]:
elem_cova = {'H': 31.0,'He': 28.0,'Li': 128.0, 'Be': 96.0, 'B': 84.0, 'C': 76.0, 'N': 71.0, 'O': 66.0, 'F': 57.0, 'Ne': 58.0, 'Na': 166.0,
 'Mg': 141.0, 'Al': 121.0, 'Si': 111.0, 'P': 107.0, 'S': 105.0, 'Cl': 102.0, 'Ar': 106.0, 'K': 203.0, 'Ca': 176.0, 'Sc': 170.0, 'Ti': 160.0, 'V': 153.0, 'Cr': 139.0, 'Mn': 139.0, 'Fe': 132.0, 'Co': 126.0, 'Ni': 124.0, 'Cu': 132.0, 'Zn': 122.0, 'Ga': 122.0, 'Ge': 120.0, 'As': 119.0, 'Se': 120.0, 'Br': 120.0, 'Kr': 116.0, 'Rb': 220.0, 'Sr': 195.0, 'Y': 190.0, 'Zr': 175.0, 'Nb': 164.0, 'Mo': 154.0, 'Tc': 147.0, 'Ru': 146.0, 'Rh': 142.0, 'Pd': 139.0, 'Ag': 145.0, 'Cd': 144.0, 'In': 142.0, 'Sn': 139.0, 'Sb': 139.0, 'Te': 138.0, 'I': 139.0, 'Xe': 140.0, 'Cs': 244.0, 'Ba': 215.0, 'La': 207.0, 'Ce': 204.0, 'Pr': 203.0, 'Nd': 201.0, 'Pm': 199.0, 'Sm': 198.0, 'Eu': 198.0, 'Gd': 196.0, 'Tb': 194.0, 'Dy': 192.0, 'Ho': 192.0, 'Er': 189.0, 'Tm': 190.0, 'Yb': 187.0, 'Lu': 187.0, 'Hf': 175.0, 'Ta': 170.0, 'W': 162.0, 'Re': 151.0, 'Os': 144.0, 'Ir': 141.0, 'Pt': 136.0, 'Au': 136.0, 'Hg': 132.0, 'Tl': 145.0, 'Pb': 146.0, 'Bi': 148.0,
 'Po': 140.0, 'At': 150.0, 'Rn': 150.0, 'Fr': 260.0, 'Ra': 221.0, 'Ac': 215.0, 'Th': 206.0, 'Pa': 200.0, 'U': 196.0, 'Np': 190.0, 'Pu': 187.0, 'Am': 180.0, 'Cm': 169.0}
cova = []
_ = pos("../CONTCAR")
for i in _.lst_atyp:
    #print(i,elem_cova[str(i)]/100)
    cova.append(elem_cova[str(i)]/100)
cova.sort()
rw_req = cova[0]+cova[1]

In [16]:
flag = 0
rw_req = rw_req
s = 1
while flag == 0:
    _ = pos("../CONTCAR")
    _.mutation()
    rw_loc = _.loc_atom
    rw_dis = []
    for i in range(len(rw_loc)-1):
        for j in range(i):
            rw_distance = (rw_loc[i][0] - rw_loc[i+1][0])**2 + (rw_loc[i][1]-rw_loc[i+1][1])**2 + (rw_loc[i][2]-rw_loc[i+1][2])**2
            #print("a,b:",i,j,rw_distance)
            rw_dis.append(rw_distance)
    rw_min = min(rw_dis)
    s = s+1
    print("test_",s)
    if rw_min > rw_req:
        flag = 1
        _.output("./POSCAR")
        _.output(topath+"/POSCAR")
        

    

test_ 2
test_ 3
test_ 4
test_ 5
test_ 6
test_ 7
test_ 8
test_ 9
test_ 10
test_ 11
test_ 12
test_ 13
test_ 14
test_ 15
test_ 16
test_ 17
test_ 18
test_ 19
test_ 20
test_ 21
test_ 22
test_ 23
test_ 24
test_ 25
test_ 26
test_ 27
test_ 28
test_ 29
test_ 30
test_ 31
test_ 32
test_ 33
test_ 34
test_ 35
test_ 36
test_ 37
test_ 38
test_ 39
test_ 40
test_ 41
test_ 42
test_ 43
test_ 44
test_ 45
test_ 46
test_ 47
test_ 48
test_ 49
test_ 50
test_ 51
test_ 52
test_ 53
test_ 54
test_ 55
test_ 56
test_ 57
test_ 58
test_ 59
test_ 60
test_ 61
test_ 62
test_ 63
test_ 64
test_ 65
test_ 66
test_ 67
test_ 68
test_ 69
test_ 70
test_ 71
test_ 72
test_ 73
test_ 74
test_ 75
test_ 76
test_ 77
test_ 78
test_ 79
test_ 80
test_ 81
test_ 82
test_ 83
test_ 84
test_ 85
test_ 86
test_ 87
test_ 88
test_ 89
test_ 90
test_ 91
test_ 92
test_ 93
test_ 94
test_ 95
test_ 96
test_ 97
test_ 98
test_ 99
test_ 100
test_ 101
test_ 102
test_ 103
test_ 104
test_ 105
test_ 106
test_ 107
test_ 108
test_ 109
test_ 110
test_ 111
test_ 

In [13]:

flag = 0
rw_req = 15
s = 1
while flag == 0:
    _ = pos("../CONTCAR")
    _.mutation()
    rw_loc = _.loc_atom
    rw_dis = []
    for i in range(len(rw_loc)-1):
        for j in range(i):
            rw_distance = (rw_loc[i][0] - rw_loc[i+1][0])**2 + (rw_loc[i][1]-rw_loc[i+1][1])**2 + (rw_loc[i][2]-rw_loc[i+1][2])**2
                #print("a,b:",i,j,rw_distance)
            rw_dis.append(rw_distance)
    rw_max = max(rw_dis)
    s = s+1
    print("test_",s)
    if rw_max < rw_req:
        flag = 1
        _.output("./POSCAR")

test_ 2
test_ 3
test_ 4
test_ 5
test_ 6
test_ 7
test_ 8
test_ 9
test_ 10


In [ ]:
_.mutation() 
_.output("./POSCAR")